In [82]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle
import numpy as np
import pandas as pd

In [112]:
d = {'strony':['https://www.aliortfi.com/dl/28745/2d43a2/KIID_ALIOR_Obligacji_kat.A.26.09.2022.pdf','https://www.aliortfi.com/dl/28746/94a732/KIID_ALIOR_Obligacji_kat.B.26.09.2022.pdf'],
     'tekst':["W co inwestujemy: Głównym składnikiem lokat Subfunduszu będą dłużne papiery wartościowe lub inne papiery wartościowe czy prawa majątkowe, o zbliżonym profilu ryzyka, a także jednostki uczestnictwa funduszy inwestycyjnych otwartych lub tytuły uczestnictwa emitowane przez fundusze zagraniczne lub instytucje wspólnego inwestowania z siedzibą za granicą.Subfundusz stosuje następujące zasady dywersyfikacji lokat: dłużne papiery wartościowe emitowane przez przedsiębiorstwa, Instrumenty Rynku Pieniężnego oraz instrumenty dłużne emitowane, gwarantowane lub poręczane przez Skarb Państwa, Narodowy Bank Polski, jednostki samorządu terytorialnego, państwa członkowskie, państwa należące do OECD, Europejski Bank Centralny, Międzynarodowy Fundusz Walutowy oraz European Stability Mechanism (w tym także dawne European Financial Stability Facility i European Financial Stabilisation Mechanism) - od 0% do 100% wartości Aktywów Subfunduszu,jednostki uczestnictwa i certyfikaty inwestycyjne funduszy inwestycyjnych, tytuły uczestnictwa funduszy zagranicznych lub innych wspólnego inwestowania z siedzibą za granicą - od 0% do 90% wartości Aktywów Subfunduszu,","Fundusz może inwestować Aktywa Subfunduszu w inne instrumenty finansowe dozwolone przepisami prawa i postanowieniami Statutu. Zyski z lokat Subfunduszu (w tym dywidendy) są ponownie inwestowane, nie są wypłacane uczestnikom i służą podwyższeniu kapitału Subfunduszu. Subfundusz może także zawierać umowy mające za przedmiot instrumenty pochodne, a także nabywać inne aktywa szczegółowo wymienione w Prospekcie Informacyjnym ALIOR SFIO. Wskaźnik referencyjny: Subfundusz nie stosuje benchmarku."]}
df = pd.DataFrame(data=d)

In [113]:
df

,strony,tekst
0,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,W co inwestujemy: Głównym składnikiem lokat Su...
1,https://www.aliortfi.com/dl/28746/94a732/KIID_...,Fundusz może inwestować Aktywa Subfunduszu w i...


### Czyszczenie

In [19]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)


In [23]:
data_clean = pd.DataFrame(df.tekst.apply(round1))
data_clean['strony']=df.strony

In [107]:
data_clean.tekst[0]

'w co inwestujemy głównym składnikiem lokat subfunduszu będą dłużne papiery wartościowe lub inne papiery wartościowe czy prawa majątkowe o zbliżonym profilu ryzyka a także jednostki uczestnictwa funduszy inwestycyjnych otwartych lub tytuły uczestnictwa emitowane przez fundusze zagraniczne lub instytucje wspólnego inwestowania z siedzibą za granicąsubfundusz stosuje następujące zasady dywersyfikacji lokat dłużne papiery wartościowe emitowane przez przedsiębiorstwa instrumenty rynku pieniężnego oraz instrumenty dłużne emitowane gwarantowane lub poręczane przez skarb państwa narodowy bank polski jednostki samorządu terytorialnego państwa członkowskie państwa należące do oecd europejski bank centralny międzynarodowy fundusz walutowy oraz european stability mechanism w tym także dawne european financial stability facility i european financial stabilisation mechanism  od  do  wartości aktywów subfunduszujednostki uczestnictwa i certyfikaty inwestycyjne funduszy inwestycyjnych tytuły uczestni

In [29]:
data_clean.to_pickle("corpus.pkl")

### Tabela z rodzielonymi słowami

In [122]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.tekst)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm['strony']=df.strony

In [123]:
data_dtm

,aktywa,aktywów,alior,bank,benchmarku,będą,centralny,certyfikaty,czy,członkowskie,...,wymienione,wypłacane,za,zagraniczne,zagranicznych,zasady,zawierać,zbliżonym,zyski,strony
0,0,2,0,2,0,1,1,1,1,1,...,0,0,2,1,1,1,0,1,0,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...
1,2,0,1,0,1,0,0,0,0,0,...,1,1,1,0,0,0,1,0,1,https://www.aliortfi.com/dl/28746/94a732/KIID_...


### Ramka surowa bagofwords

In [60]:
data_dtm2= data_dtm.melt(id_vars=['strony'])
data_dtm2

,strony,variable,value
0,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,aktywa,0
1,https://www.aliortfi.com/dl/28746/94a732/KIID_...,aktywa,2
2,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,aktywów,2
3,https://www.aliortfi.com/dl/28746/94a732/KIID_...,aktywów,0
4,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,alior,0
...,...,...,...
219,https://www.aliortfi.com/dl/28746/94a732/KIID_...,zawierać,1
220,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zbliżonym,1
221,https://www.aliortfi.com/dl/28746/94a732/KIID_...,zbliżonym,0
222,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zyski,0


In [61]:
data_dtm2.columns = ["strony","słowa","liczność"]

In [63]:
data_dtm2 = data_dtm2[data_dtm2.liczność != 0]

In [64]:
data_dtm2

,strony,słowa,liczność
1,https://www.aliortfi.com/dl/28746/94a732/KIID_...,aktywa,2
2,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,aktywów,2
5,https://www.aliortfi.com/dl/28746/94a732/KIID_...,alior,1
6,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,bank,2
9,https://www.aliortfi.com/dl/28746/94a732/KIID_...,benchmarku,1
...,...,...,...
214,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zagranicznych,1
216,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zasady,1
219,https://www.aliortfi.com/dl/28746/94a732/KIID_...,zawierać,1
220,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zbliżonym,1


### Ramka znormalizowana bagofwords

In [67]:
from stempel import StempelStemmer
stemmer = StempelStemmer.polimorf()

Loading: 100%|███████████████| 11368252/11368252 [00:18<00:00, 600044.26bytes/s]


In [124]:
list_stemmer = []
data_dtm.drop(axis=1,columns=['strony'])
for word in data_dtm.columns :
    list_stemmer.append(stemmer.stem(word))

In [108]:
data_dtm.columns = list_stemmer

In [125]:
data_dtm

,aktywa,aktywów,alior,bank,benchmarku,będą,centralny,certyfikaty,czy,członkowskie,...,wymienione,wypłacane,za,zagraniczne,zagranicznych,zasady,zawierać,zbliżonym,zyski,strony
0,0,2,0,2,0,1,1,1,1,1,...,0,0,2,1,1,1,0,1,0,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...
1,2,0,1,0,1,0,0,0,0,0,...,1,1,1,0,0,0,1,0,1,https://www.aliortfi.com/dl/28746/94a732/KIID_...


In [126]:
data_dtm= data_dtm.groupby(by=data_dtm.columns, axis=1).sum()

In [127]:
data_dtm['strony']=df.strony
data_dtm3= data_dtm.melt(id_vars=['strony'])
data_dtm3.columns = ["nazwa funduszu/subfunduszu","słowa","liczność"]
data_dtm3

,nazwa funduszu/subfunduszu,słowa,liczność
0,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,aktywa,0
1,https://www.aliortfi.com/dl/28746/94a732/KIID_...,aktywa,2
2,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,aktywów,2
3,https://www.aliortfi.com/dl/28746/94a732/KIID_...,aktywów,0
4,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,alior,0
...,...,...,...
219,https://www.aliortfi.com/dl/28746/94a732/KIID_...,zawierać,1
220,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zbliżonym,1
221,https://www.aliortfi.com/dl/28746/94a732/KIID_...,zbliżonym,0
222,https://www.aliortfi.com/dl/28745/2d43a2/KIID_...,zyski,0


In [131]:
import pdfplumber

In [133]:
pdf = pdfplumber.open("KIID_ALIOR_Obligacji_kat.A.26.09.2022.pdf")

In [134]:
p0 = pdf.pages[0]

In [135]:
text = p0.extract_text()
print(text)

Kluczowe Informacje dla Inwestorów
Niniejszy dokument zawiera kluczowe informacje dla Inwestorów dotyczące tego funduszu. Nie są to
materiały marketingowe. Dostarczenie tych informacji jest wymogiem prawnym mającym na celu ułatwienie
zrozumienia charakteru i ryzyka związanego z inwestowaniem w ten fundusz. Przeczytanie niniejszego
dokumentu jest zalecane inwestorowi, aby mógł on podjąć świadomą decyzję inwestycyjną.
ALIOR Obligacji, Subfundusz wydzielony w ramach ALIOR Specjalistycznego Funduszu
Inwestycyjnego Otwartego (ALIOR SFIO),
kategoria jednostek uczestnictwa: A
Spółka zarządzająca: Alior TFI S.A., członek grupy Alior Bank
 
Cele i polityka inwestycyjna
zagraniczne.
W co inwestujemy:  Głównym  składnikiem  lokat  Subfunduszu  będą
dłużne papiery wartościowe lub inne papiery wartościowe czy prawa Do  100%  wartości  Aktywów  Subfunduszu  mogą  stanowić  aktywa
majątkowe, o zbliżonym profilu ryzyka, a także jednostki uczestnictwa denominowane w walutach obcych.
funduszy inwestycyj